In [ ]:
%pip install numpy
%pip install matplotlib

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from typing import List, Tuple, Union
from math import ceil, floor, sqrt

## Setup

### Helper functions

In [2]:
def split_data(points: np.ndarray, labels: np.ndarray, train_size: Union[int, float], test_size: Union[int, float]):
    """Split data into train and test datasets
    """
    number_of_points = points.shape[0]

    if isinstance(train_size, float) and isinstance(test_size, float):
        if (train_size + test_size) != 1.0:
            raise ValueError(f"train size + test size should be equal to 1.0")

        train_size = ceil(train_size * number_of_points)
        test_size = floor(test_size * number_of_points)
    elif isinstance(train_size, int) and isinstance(test_size, int):
        if number_of_points != (train_size + test_size):
            raise ValueError(f"train size + test size should be equal to the number of points")
    else:
        raise TypeError("train_size and test_size should be the same type")

    splitted_points = np.split(points, [train_size, number_of_points], axis=0)[:-1]
    splitted_labels = np.split(labels, [train_size, number_of_points], axis=0)[:-1]
    return splitted_points + splitted_labels

### Data generator

In [30]:
class Data:
    _DATA_RANDOM_SEED: int = 8765
    _data_random_generator: np.random.Generator = np.random.default_rng(seed=5678)

    # deterministic random generator for data
    @classmethod
    def get_data_random_generator(cls, deterministic: bool = True) -> np.random.Generator:
        global _data_random_generator
        if deterministic:
            return np.random.default_rng(seed=cls._DATA_RANDOM_SEED)
        else:
            return cls._data_random_generator

    @classmethod
    def generate_uniform_data(cls, n: int):
        points = cls.get_data_random_generator().uniform(low=0, high=1, size=(n, 2))
        labels = np.fromiter(map(lambda point: int(point[0] <= point[1]), points), dtype=int)
        return points, np.expand_dims(labels, axis=-1)

    @classmethod
    def generate_X_like_data(cls):
        points = []
        labels = []
        for i in range(11):
            value = 0.1 * i
            points.append([value, value])
            labels.append(0)

            if value == 0.5:
                continue

            points.append([value, 1 - value])
            labels.append(1)
        return np.array(points, dtype=float), np.expand_dims(np.array(labels, dtype=int), axis=-1)

### Network components

In [153]:
class Module:
    def reset_parameters(self) -> None:
        pass

    def forward(self, x: np.ndarray) -> np.ndarray:
        return None

    def backward(self) -> None:
        pass

class Sigmoid(Module):
    def __init__(self) -> None:
        super(Sigmoid, self).__init__()
        pass

    def forward(self, x: np.ndarray) -> np.ndarray:
        return 1.0 / (1.0 + np.exp(-x))

    def backward(self) -> None:
        pass

class Dense(Module):
    _random_generator: np.random.Generator
    _input_feature_size: int
    _output_feature_size: int
    _weight: np.ndarray
    _bias: np.ndarray

    def __init__(self, input_feature_size: int, output_feature_size: int, bias: bool = True, random_generator: np.random.Generator = np.random.default_rng()) -> None:
        super(Dense, self).__init__()
        self._random_generator = random_generator
        self._input_feature_size = input_feature_size
        self._output_feature_size = output_feature_size
        self._weight = np.zeros((output_feature_size, input_feature_size))
        if bias:
            self._bias = np.zeros(output_feature_size)
        else:
            self._bias = None

    def reset_parameters(self) -> None:
        random_range = sqrt(1 / self._input_feature_size)
        self._weight = self._random_generator.uniform(low = -random_range, high = random_range, size = (self._output_feature_size, self._input_feature_size))
        self._bias = self._random_generator.uniform(low = -random_range, high = random_range, size = self._output_feature_size)

    def forward(self, x: np.ndarray) -> np.ndarray:
        # @ is equivalent to np.matmul, + is equivalent to np.add
        return x @ self._weight.swapaxes(-1, -2) + self._bias

    def backward(self) -> None:
        pass

class Network(Module):
    _layers: List[Module]

    def __init__(self, input_size: int, output_size: int, hidden_sizes: Union[List[int], Tuple[int]], random_generator: np.random.Generator) -> None:
        super(Network, self).__init__()
        self._layers = []

        for hidden_size in hidden_sizes:
            layer = Dense(input_size, hidden_size, random_generator = random_generator)
            activation_layer = Sigmoid()
            self._layers.extend([layer, activation_layer])
            input_size = hidden_size

        output_layer = Dense(input_size, output_size, random_generator = random_generator)
        self._layers.extend([output_layer, Sigmoid()])

        for layer in self._layers:
            layer.reset_parameters()

    def forward(self, x: np.ndarray) -> np.ndarray:
        y = x
        for layer in self._layers:
            y = layer.forward(y)
        return y

    def backward(self) -> None:
        pass

In [156]:
model_random_generator: np.random.Generator = np.random.default_rng(seed=7777)
x = Data.get_data_random_generator(False).normal(size=(10, 5))

network = Network(x.shape[-1], 1, (5, 3), model_random_generator)
network.forward(x)

array([[0.43431755],
       [0.42743879],
       [0.43155144],
       [0.42360968],
       [0.42772034],
       [0.42423286],
       [0.42459648],
       [0.43097408],
       [0.43207898],
       [0.42214131]])

## Results

### Uniform distribution

In [24]:
points, labels = Data.generate_uniform_data(500)
x_train, x_validation, y_train, y_validation = split_data(points, labels, 0.67, 0.33)
x_test, y_test = Data.generate_uniform_data(100)

print("x train data: ", x_train[:5])
print("y train data: ", y_train[:5])

print("shape of train data:", x_train.shape)
print("shape of validation data:", x_validation.shape)
print("shape of validation data:", x_test.shape)

x train data:  [[0.50779375 0.98285872]
 [0.62990052 0.32851095]
 [0.33831504 0.6298731 ]
 [0.13917241 0.81296814]
 [0.48371171 0.47093232]]
y train data:  [[1]
 [0]
 [1]
 [1]
 [0]]
shape of train data: (335, 2)
shape of validation data: (165, 2)
shape of validation data: (100, 2)


### X like

In [77]:
x, y = Data.generate_X_like_data()